# 🧪 Tests Pratiques et Comparaisons - Évaluation Réelle

## 🎯 Objectifs
- **Tester** les modèles sur de vraies tâches
- **Mesurer** la latence et le coût réels
- **Comparer** les performances sur vos cas d'usage
- **Valider** les recommandations théoriques

---

## 🤔 Pourquoi Tester en Pratique ?

### Les Benchmarks ne Disent Pas Tout !
- 📊 **MMLU** : Excellent sur les questions académiques
- 🎯 **Votre usage** : Peut-être différent !

### Exemple Concret :
```
GPT-4 : 86.4% MMLU → "Meilleur modèle"
Mistral 7B : 62.5% MMLU → "Modèle moyen"

MAIS sur votre tâche spécifique :
GPT-4 : Excellent mais 3 secondes et 30$/1M tokens
Mistral 7B : 90% de la qualité en 0.8 seconde et gratuit

→ Mistral 7B peut être MEILLEUR pour vous !
```

## 📦 Installation et Setup

In [1]:
# Installation des dépendances
!pip install requests pandas matplotlib seaborn plotly numpy time aiohttp asyncio

ERROR: Could not find a version that satisfies the requirement time (from versions: none)
ERROR: No matching distribution found for time


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
import time
import requests
import json
import asyncio
import aiohttp
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("✅ Imports terminés !")

ModuleNotFoundError: No module named 'aiohttp'

## 🔑 Configuration des APIs

### ⚠️ Important : Sécurité des Clés API

**JAMAIS** de clés API directement dans le code !

### 🛡️ Méthodes Sécurisées :
1. **Variables d'environnement** (recommandé)
2. **Fichier .env** (non versionné)
3. **Saisie manuelle** (pour les tests)

### 🔧 Setup Recommandé :
```bash
# Dans votre terminal :
export OPENAI_API_KEY="sk-..."
export ANTHROPIC_API_KEY="sk-ant-..."
export GOOGLE_API_KEY="AI..."
```

In [ ]:
import os
from getpass import getpass

# Configuration sécurisée des APIs
def setup_api_keys():
    """
    Configuration sécurisée des clés API
    """
    api_keys = {}
    
    # Essayer les variables d'environnement d'abord
    api_keys['openai'] = os.getenv('OPENAI_API_KEY')
    api_keys['anthropic'] = os.getenv('ANTHROPIC_API_KEY')
    api_keys['google'] = os.getenv('GOOGLE_API_KEY')
    
    # Si pas trouvées, demander à l'utilisateur
    print("🔑 Configuration des clés API")
    print("⚠️  Appuyez sur Entrée pour ignorer une API si vous n'avez pas la clé")
    
    if not api_keys['openai']:
        key = getpass("OpenAI API Key (sk-...): ")
        api_keys['openai'] = key if key.strip() else None
    
    if not api_keys['anthropic']:
        key = getpass("Anthropic API Key (sk-ant-...): ")
        api_keys['anthropic'] = key if key.strip() else None
    
    if not api_keys['google']:
        key = getpass("Google API Key (AI...): ")
        api_keys['google'] = key if key.strip() else None
    
    # Afficher les APIs disponibles
    available_apis = [k for k, v in api_keys.items() if v]
    print(f"\n✅ APIs configurées : {', '.join(available_apis)}")
    
    if not available_apis:
        print("⚠️  Aucune API configurée. Nous utiliserons des données simulées.")
    
    return api_keys

# Configurer les APIs
API_KEYS = setup_api_keys()

## 🧪 Framework de Test Unifié

### 🎯 Tâches de Test Standardisées

Nous allons tester les modèles sur 5 tâches représentatives :

1. **📝 Résumé de texte**
2. **💬 Questions-réponses**
3. **💻 Génération de code**
4. **🌍 Traduction**
5. **🎨 Créativité (storytelling)**

In [3]:
# Dataset de test standardisé
TEST_TASKS = {
    'resume': {
        'name': 'Résumé de Texte',
        'prompt': "Résumez ce texte en 2-3 phrases claires :",
        'input': """L'intelligence artificielle (IA) connaît une révolution sans précédent avec l'émergence des grands modèles de langage (LLM). Ces systèmes, entraînés sur d'énormes corpus de texte, démontrent des capacités remarquables de compréhension et de génération de langage naturel. 
        
GPT-4, Claude, et leurs concurrents transforment déjà de nombreux secteurs, de l'éducation à la programmation. Cependant, ces avancées soulèvent aussi des questions importantes sur l'éthique, l'emploi, et la véracité des informations générées. 

Les entreprises investissent massivement dans cette technologie, mais doivent naviguer entre opportunités et risques. Le choix du bon modèle devient crucial pour maximiser les bénéfices tout en contrôlant les coûts et les risques.""",
        'max_tokens': 100
    },
    
    'qa': {
        'name': 'Questions-Réponses',
        'prompt': "Répondez précisément à cette question :",
        'input': "Quels sont les 3 principaux avantages des modèles LLM open source par rapport aux modèles propriétaires ?",
        'max_tokens': 150
    },
    
    'code': {
        'name': 'Génération de Code',
        'prompt': "Écrivez une fonction Python pour :",
        'input': "Calculer la moyenne pondérée d'une liste de nombres avec leurs poids. La fonction doit gérer les cas d'erreur et retourner None si les listes ont des tailles différentes.",
        'max_tokens': 200
    },
    
    'translation': {
        'name': 'Traduction',
        'prompt': "Traduisez ce texte français en anglais naturel :",
        'input': "Bonjour, j'aimerais réserver une table pour quatre personnes ce soir vers 20 heures. Avez-vous quelque chose de disponible ? De préférence près de la fenêtre si possible.",
        'max_tokens': 100
    },
    
    'creativity': {
        'name': 'Créativité',
        'prompt': "Écrivez le début d'une histoire courte (2-3 paragraphes) avec ce thème :",
        'input': "Un développeur découvre que son IA domestique a commencé à écrire de la poésie la nuit, en secret.",
        'max_tokens': 200
    }
}

print("📋 Dataset de test créé !")
print(f"Nombre de tâches : {len(TEST_TASKS)}")
for task_id, task in TEST_TASKS.items():
    print(f"  • {task['name']} ({task['max_tokens']} tokens max)")

📋 Dataset de test créé !
Nombre de tâches : 5
  • Résumé de Texte (100 tokens max)
  • Questions-Réponses (150 tokens max)
  • Génération de Code (200 tokens max)
  • Traduction (100 tokens max)
  • Créativité (200 tokens max)


## 🔌 Connecteurs API

In [4]:
# Simulateur de réponses si pas d'API
class ModelSimulator:
    """Simule les réponses des modèles si les APIs ne sont pas disponibles"""
    
    def __init__(self, model_name):
        self.model_name = model_name
        self.responses = {
            'resume': {
                'gpt-4': "Les LLM révolutionnent l'IA avec des capacités remarquables de langage naturel. Ils transforment de nombreux secteurs mais soulèvent des questions éthiques. Le choix du bon modèle est crucial pour les entreprises.",
                'claude-3': "L'IA connaît une révolution avec les grands modèles de langage qui démontrent des capacités exceptionnelles. Ces avancées transforment les secteurs tout en soulevant des défis éthiques et économiques. Les entreprises doivent choisir leur modèle avec soin.",
                'gemini': "Les LLM marquent une révolution de l'IA avec des capacités linguistiques avancées qui transforment l'éducation et la programmation. Cependant, ils posent des défis éthiques et économiques que les entreprises doivent naviguer soigneusement."
            },
            'qa': {
                'gpt-4': "Les 3 principaux avantages des LLM open source sont : 1) Contrôle total et confidentialité des données, 2) Coûts réduits (pas de frais par API), 3) Personnalisation complète via fine-tuning.",
                'claude-3': "Les modèles open source offrent : 1) Une confidentialité totale des données traitées localement, 2) Des coûts prévisibles sans dépendance aux tarifs API, 3) Une personnalisation poussée adaptée aux besoins spécifiques.",
                'gemini': "Avantages clés de l'open source : 1) Privacy et sécurité avec traitement local, 2) Économies substantielles sur les volumes importants, 3) Flexibilité maximale pour adaptation et fine-tuning."
            }
        }
    
    def generate(self, prompt, task_type='resume'):
        """Simule une génération"""
        model_key = self.model_name.lower().replace('-', '').replace(' ', '')
        
        if 'gpt' in model_key:
            model_key = 'gpt-4'
        elif 'claude' in model_key:
            model_key = 'claude-3'
        else:
            model_key = 'gemini'
        
        response = self.responses.get(task_type, {}).get(model_key, "Réponse simulée pour cette tâche.")
        
        # Simuler la latence
        if 'gpt-4' in self.model_name.lower():
            time.sleep(2)  # GPT-4 plus lent
        elif 'mistral' in self.model_name.lower():
            time.sleep(0.5)  # Mistral plus rapide
        else:
            time.sleep(1)  # Autres modèles
        
        return response

# Fonction de test unifiée
async def test_model_on_task(model_name, task_id, task_data, api_keys=None):
    """
    Teste un modèle sur une tâche spécifique
    """
    prompt = f"{task_data['prompt']}\n\n{task_data['input']}\n\nRéponse:"
    
    # Mesurer le temps de début
    start_time = time.time()
    
    try:
        # Si APIs disponibles, utiliser les vraies APIs
        if api_keys and any(api_keys.values()):
            # Ici on pourrait implémenter les vraies APIs
            # Pour ce demo, on utilise le simulateur
            simulator = ModelSimulator(model_name)
            response = simulator.generate(prompt, task_id)
        else:
            # Utiliser le simulateur
            simulator = ModelSimulator(model_name)
            response = simulator.generate(prompt, task_id)
        
        # Calculer la latence
        latency = time.time() - start_time
        
        # Calculer le coût (simulation)
        input_tokens = len(prompt.split()) * 1.3  # Approximation
        output_tokens = len(response.split()) * 1.3
        total_tokens = input_tokens + output_tokens
        
        # Coûts par modèle ($/1M tokens)
        costs_per_million = {
            'GPT-4': 30.0,
            'GPT-3.5 Turbo': 1.0,
            'Claude 3 Opus': 15.0,
            'Claude 3 Sonnet': 3.0,
            'Gemini Pro': 2.5,
            'Mistral 7B': 0.0,  # Open source
            'Llama 2': 0.0     # Open source
        }
        
        cost_per_token = costs_per_million.get(model_name, 0) / 1_000_000
        cost = total_tokens * cost_per_token
        
        return {
            'model': model_name,
            'task': task_id,
            'task_name': task_data['name'],
            'response': response,
            'latency': latency,
            'input_tokens': int(input_tokens),
            'output_tokens': int(output_tokens),
            'total_tokens': int(total_tokens),
            'cost': cost,
            'success': True
        }
    
    except Exception as e:
        return {
            'model': model_name,
            'task': task_id,
            'task_name': task_data['name'],
            'response': f"Erreur: {str(e)}",
            'latency': None,
            'input_tokens': 0,
            'output_tokens': 0,
            'total_tokens': 0,
            'cost': 0,
            'success': False
        }

print("🔌 Framework de test configuré !")

🔌 Framework de test configuré !


## 🏁 Lancement des Tests

### 🎯 Modèles à Tester

Nous allons comparer 6 modèles représentatifs :

In [5]:
# Modèles à tester
MODELS_TO_TEST = [
    'GPT-4',
    'GPT-3.5 Turbo', 
    'Claude 3 Opus',
    'Claude 3 Sonnet',
    'Gemini Pro',
    'Mistral 7B'
]

print(f"🧪 Lancement des tests sur {len(MODELS_TO_TEST)} modèles et {len(TEST_TASKS)} tâches")
print(f"📊 Total de tests : {len(MODELS_TO_TEST) * len(TEST_TASKS)} = {len(MODELS_TO_TEST) * len(TEST_TASKS)}")
print("\n⏱️  Temps estimé : 2-3 minutes (avec simulation)")
print("\n🚀 Démarrage des tests...")

# Lancer tous les tests
test_results = []

for i, model in enumerate(MODELS_TO_TEST, 1):
    print(f"\n📋 Test du modèle {i}/{len(MODELS_TO_TEST)}: {model}")
    
    for j, (task_id, task_data) in enumerate(TEST_TASKS.items(), 1):
        print(f"  🧪 Tâche {j}/{len(TEST_TASKS)}: {task_data['name']}... ", end="")
        
        # Test synchrone pour la démo
        result = await test_model_on_task(model, task_id, task_data, API_KEYS)
        test_results.append(result)
        
        if result['success']:
            print(f"✅ {result['latency']:.2f}s, {result['total_tokens']} tokens")
        else:
            print(f"❌ Erreur")

print(f"\n🎉 Tests terminés ! {len(test_results)} résultats collectés.")

# Convertir en DataFrame pour l'analyse
df_results = pd.DataFrame(test_results)
df_results.head()

🧪 Lancement des tests sur 6 modèles et 5 tâches
📊 Total de tests : 30 = 30

⏱️  Temps estimé : 2-3 minutes (avec simulation)

🚀 Démarrage des tests...

📋 Test du modèle 1/6: GPT-4
  🧪 Tâche 1/5: Résumé de Texte... 

NameError: name 'API_KEYS' is not defined

## 📊 Analyse des Résultats

### ⚡ Performance vs Latence

In [6]:
# Analyser la latence par modèle
latency_by_model = df_results[df_results['success']].groupby('model')['latency'].agg(['mean', 'std']).reset_index()
latency_by_model.columns = ['Model', 'Latence_Moyenne', 'Latence_Std']

# Graphique de latence
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Latence moyenne par modèle
bars = ax1.bar(latency_by_model['Model'], latency_by_model['Latence_Moyenne'], 
               yerr=latency_by_model['Latence_Std'], capsize=5,
               color=['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4', '#FECA57', '#FF9FF3'])

ax1.set_title('⚡ Latence Moyenne par Modèle', fontweight='bold', fontsize=14)
ax1.set_ylabel('Latence (secondes)')
ax1.tick_params(axis='x', rotation=45)
ax1.grid(True, alpha=0.3, axis='y')

# Ajouter les valeurs sur les barres
for bar, latence in zip(bars, latency_by_model['Latence_Moyenne']):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.05, 
             f'{latence:.2f}s', ha='center', fontweight='bold')

# Latence par tâche
latency_by_task = df_results[df_results['success']].groupby('task_name')['latency'].mean().reset_index()
latency_by_task = latency_by_task.sort_values('latency')

ax2.barh(latency_by_task['task_name'], latency_by_task['latency'],
         color=['#E74C3C', '#3498DB', '#2ECC71', '#F39C12', '#9B59B6'])
ax2.set_title('⏱️ Latence Moyenne par Tâche', fontweight='bold', fontsize=14)
ax2.set_xlabel('Latence (secondes)')
ax2.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

print("🏆 Classement Vitesse (plus rapide → plus lent):")
speed_ranking = latency_by_model.sort_values('Latence_Moyenne')
for i, (_, row) in enumerate(speed_ranking.iterrows(), 1):
    medal = ['🥇', '🥈', '🥉', '4️⃣', '5️⃣', '6️⃣'][i-1]
    print(f"  {medal} {row['Model']:15s}: {row['Latence_Moyenne']:.2f}s (±{row['Latence_Std']:.2f}s)")

NameError: name 'df_results' is not defined

## 💰 Analyse des Coûts Réels

In [ ]:
# Analyser les coûts
cost_analysis = df_results[df_results['success']].groupby('model').agg({
    'cost': ['sum', 'mean'],
    'total_tokens': ['sum', 'mean']
}).round(6)

cost_analysis.columns = ['Coût_Total', 'Coût_Moyen', 'Tokens_Total', 'Tokens_Moyen']
cost_analysis = cost_analysis.reset_index()

# Projeter les coûts sur différents volumes
volumes = [1_000, 10_000, 100_000, 1_000_000]  # Nombre de requêtes par mois
volume_labels = ['1K req/mois', '10K req/mois', '100K req/mois', '1M req/mois']

# Calculer les coûts projetés
cost_projections = []
for _, row in cost_analysis.iterrows():
    for vol, vol_label in zip(volumes, volume_labels):
        monthly_cost = row['Coût_Moyen'] * vol
        cost_projections.append({
            'Model': row['model'],
            'Volume': vol_label,
            'Volume_Num': vol,
            'Coût_Mensuel': monthly_cost
        })

df_cost_proj = pd.DataFrame(cost_projections)

# Graphique des coûts projetés
fig = px.bar(df_cost_proj, x='Volume', y='Coût_Mensuel', color='Model',
             title='💰 Projection des Coûts Mensuels par Volume d\'Usage',
             labels={'Coût_Mensuel': 'Coût Mensuel ($)', 'Volume': 'Volume d\'Usage'},
             height=500)

# Échelle logarithmique pour mieux voir les différences
fig.update_layout(yaxis_type="log")
fig.show()

# Tableau détaillé des coûts
print("💵 Analyse Détaillée des Coûts (pour les 5 tâches de test):")
print("=" * 70)
for _, row in cost_analysis.iterrows():
    print(f"\n📊 {row['model']}:")
    print(f"  💰 Coût par test: ${row['Coût_Moyen']:.6f}")
    print(f"  🔢 Tokens moyens: {row['Tokens_Moyen']:.0f}")
    print(f"  📈 Coût 1K requêtes/mois: ${row['Coût_Moyen'] * 1000:.2f}")
    print(f"  📈 Coût 100K requêtes/mois: ${row['Coût_Moyen'] * 100000:.2f}")

# Point de bascule coût
print("\n🎯 Points de Bascule Économiques:")
proprietary_models = ['GPT-4', 'GPT-3.5 Turbo', 'Claude 3 Opus', 'Claude 3 Sonnet', 'Gemini Pro']
open_source_hosting_cost = 500  # $/mois estimation hébergement

for model in proprietary_models:
    model_cost = cost_analysis[cost_analysis['model'] == model]['Coût_Moyen'].iloc[0]
    if model_cost > 0:
        breakeven_requests = open_source_hosting_cost / model_cost
        print(f"  📊 {model}: Bascule vers open source à {breakeven_requests:,.0f} requêtes/mois")

## 🎯 Évaluation Qualitative

### 📝 Comparaison des Réponses

In [ ]:
# Fonction d'évaluation qualitative simple
def evaluate_response_quality(response, task_type):
    """
    Évaluation qualitative basique d'une réponse
    (Dans un vrai projet, utilisez des métriques plus sophistiquées)
    """
    if not response or response.startswith("Erreur"):
        return 0
    
    score = 50  # Score de base
    
    # Critères généraux
    if len(response) > 20:  # Réponse substantielle
        score += 20
    
    if '.' in response and len(response.split('.')) > 1:  # Structure
        score += 10
    
    # Critères spécifiques par tâche
    if task_type == 'code':
        if 'def ' in response:  # Fonction définie
            score += 15
        if 'return' in response:  # Return statement
            score += 10
    
    elif task_type == 'resume':
        words = len(response.split())
        if 20 <= words <= 60:  # Longueur appropriée
            score += 15
    
    elif task_type == 'translation':
        if any(word in response.lower() for word in ['hello', 'table', 'tonight', 'available']):
            score += 15
    
    return min(score, 100)  # Cap à 100

# Calculer les scores de qualité
df_results['quality_score'] = df_results.apply(
    lambda row: evaluate_response_quality(row['response'], row['task']) if row['success'] else 0, 
    axis=1
)

# Analyser la qualité par modèle et tâche
quality_by_model_task = df_results[df_results['success']].pivot_table(
    index='model', 
    columns='task_name', 
    values='quality_score', 
    aggfunc='mean'
).fillna(0)

# Heatmap de qualité
plt.figure(figsize=(12, 8))
sns.heatmap(quality_by_model_task, 
            annot=True, 
            cmap='RdYlGn', 
            center=75,
            fmt='.0f',
            cbar_kws={'label': 'Score de Qualité'})
plt.title('🎯 Qualité des Réponses par Modèle et Tâche', fontweight='bold', fontsize=16)
plt.ylabel('Modèle')
plt.xlabel('Tâche')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

# Score de qualité moyen par modèle
quality_by_model = df_results[df_results['success']].groupby('model')['quality_score'].mean().sort_values(ascending=False)

print("🏆 Classement Qualité (basé sur évaluation automatique):")
for i, (model, score) in enumerate(quality_by_model.items(), 1):
    medal = ['🥇', '🥈', '🥉', '4️⃣', '5️⃣', '6️⃣'][i-1]
    print(f"  {medal} {model:15s}: {score:.0f}/100")

print("\n⚠️  Note : Ces scores sont basés sur une évaluation automatique simple.")
print("   Pour une évaluation rigoureuse, utilisez des métriques spécialisées et l'évaluation humaine.")

## 📈 Synthèse : Rapport Qualité/Prix/Vitesse

In [ ]:
# Créer un score composite
summary_stats = df_results[df_results['success']].groupby('model').agg({
    'quality_score': 'mean',
    'latency': 'mean',
    'cost': 'mean'
}).reset_index()

# Normaliser les métriques (0-100)
summary_stats['quality_norm'] = summary_stats['quality_score']  # Déjà 0-100
summary_stats['speed_norm'] = 100 - (summary_stats['latency'] / summary_stats['latency'].max() * 100)  # Inverse latence

# Pour le coût, différencier modèles payants vs gratuits
max_cost = summary_stats[summary_stats['cost'] > 0]['cost'].max()
summary_stats['cost_norm'] = summary_stats.apply(
    lambda row: 100 if row['cost'] == 0 else 100 - (row['cost'] / max_cost * 80),  # Gratuit = 100, max payant = 20
    axis=1
)

# Calculer des scores composites pour différents profils
profiles = {
    'Qualité Max': {'quality': 0.7, 'speed': 0.1, 'cost': 0.2},
    'Équilibré': {'quality': 0.4, 'speed': 0.3, 'cost': 0.3},
    'Budget Limité': {'quality': 0.2, 'speed': 0.2, 'cost': 0.6},
    'Temps Réel': {'quality': 0.3, 'speed': 0.6, 'cost': 0.1}
}

for profile_name, weights in profiles.items():
    summary_stats[f'score_{profile_name.lower().replace(" ", "_")}'] = (
        summary_stats['quality_norm'] * weights['quality'] +
        summary_stats['speed_norm'] * weights['speed'] +
        summary_stats['cost_norm'] * weights['cost']
    )

# Graphique radar pour chaque profil
fig = make_subplots(
    rows=2, cols=2,
    specs=[[{"type": "polar"}, {"type": "polar"}],
           [{"type": "polar"}, {"type": "polar"}]],
    subplot_titles=list(profiles.keys())
)

positions = [(1, 1), (1, 2), (2, 1), (2, 2)]
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4', '#FECA57', '#FF9FF3']

for i, (profile_name, _) in enumerate(profiles.items()):
    row, col = positions[i]
    
    # Top 4 modèles pour ce profil
    score_col = f'score_{profile_name.lower().replace(" ", "_")}'
    top_models = summary_stats.nlargest(4, score_col)
    
    for j, (_, model_row) in enumerate(top_models.iterrows()):
        values = [model_row['quality_norm'], model_row['speed_norm'], model_row['cost_norm']]
        
        fig.add_trace(
            go.Scatterpolar(
                r=values + [values[0]],
                theta=['Qualité', 'Vitesse', 'Coût'] + ['Qualité'],
                fill='toself',
                name=model_row['model'],
                line_color=colors[j],
                opacity=0.6
            ),
            row=row, col=col
        )

fig.update_layout(
    title="🎯 Recommandations par Profil d'Usage (Top 4 par profil)",
    height=800,
    showlegend=False
)

# Configurer les axes polaires
for i in range(1, 3):
    for j in range(1, 3):
        fig.update_polars(
            radialaxis=dict(visible=True, range=[0, 100]),
            row=i, col=j
        )

fig.show()

# Recommandations textuelles
print("🎯 RECOMMANDATIONS FINALES")
print("=" * 50)

for profile_name, weights in profiles.items():
    score_col = f'score_{profile_name.lower().replace(" ", "_")}'
    best_model = summary_stats.loc[summary_stats[score_col].idxmax()]
    
    print(f"\n🏆 {profile_name}:")
    print(f"  Recommandé: {best_model['model']}")
    print(f"  Score global: {best_model[score_col]:.1f}/100")
    print(f"  Qualité: {best_model['quality_norm']:.0f}/100")
    print(f"  Vitesse: {best_model['speed_norm']:.0f}/100")
    print(f"  Coût: {best_model['cost_norm']:.0f}/100")
    print(f"  Latence réelle: {best_model['latency']:.2f}s")
    print(f"  Coût réel/test: ${best_model['cost']:.6f}")

## 📝 Exemple de Réponses Détaillées

### 🔍 Comparaison Qualitative sur la Tâche "Résumé"

In [ ]:
# Afficher les réponses pour la tâche de résumé
resume_results = df_results[(df_results['task'] == 'resume') & (df_results['success'])]

print("📝 COMPARAISON DES RÉSUMÉS")
print("=" * 60)
print("\n📖 Texte original (extrait):")
print(TEST_TASKS['resume']['input'][:200] + "...")
print("\n🎯 Consigne: Résumez en 2-3 phrases claires")
print("\n" + "=" * 60)

for _, result in resume_results.iterrows():
    print(f"\n🤖 {result['model']}:")
    print(f"⏱️  Latence: {result['latency']:.2f}s")
    print(f"💰 Coût: ${result['cost']:.6f}")
    print(f"📊 Score: {result['quality_score']:.0f}/100")
    print(f"📝 Réponse:")
    print(f"   {result['response']}")
    print("-" * 40)

# Analyse comparative
print("\n🔍 ANALYSE COMPARATIVE:")
resume_analysis = resume_results.groupby('model').agg({
    'latency': 'first',
    'cost': 'first', 
    'quality_score': 'first',
    'response': lambda x: len(x.iloc[0].split())  # Nombre de mots
}).round(3)
resume_analysis.columns = ['Latence (s)', 'Coût ($)', 'Score Qualité', 'Nb Mots']

print(resume_analysis.sort_values('Score Qualité', ascending=False))

print("\n💡 Observations:")
fastest = resume_results.loc[resume_results['latency'].idxmin()]['model']
cheapest = resume_results.loc[resume_results['cost'].idxmin()]['model'] 
best_quality = resume_results.loc[resume_results['quality_score'].idxmax()]['model']

print(f"  ⚡ Plus rapide: {fastest}")
print(f"  💰 Moins cher: {cheapest}")
print(f"  🏆 Meilleure qualité: {best_quality}")

## 🎯 Conclusions et Recommandations Pratiques

### ✅ Points Clés des Tests Réels

1. **📊 Les benchmarks ne racontent qu'une partie de l'histoire**
   - Performance réelle dépend de votre cas d'usage
   - Latence varie énormément selon le modèle
   - Coûts peuvent exploser selon le volume

2. **⚡ Vitesse vs Qualité : Compromis inévitable**
   - Modèles rapides souvent "assez bons"
   - Modèles lents excellents pour tâches critiques
   - Identifier votre seuil de tolérance

3. **💰 Coût réel ≠ Prix annoncé**
   - Compter les tokens d'input ET output
   - Volume change complètement l'équation
   - Modèles gratuits ont coûts d'hébergement

4. **🎯 Pas de modèle parfait universel**
   - Chaque profil d'usage a son optimal
   - Tester sur VOS données = indispensable
   - Réévaluer régulièrement (nouveaux modèles)

### 🛠️ Méthodologie de Test Recommandée

1. **📋 Définir vos tâches représentatives**
   - 3-5 tâches typiques de votre usage
   - Avec vos données réelles
   - Métriques de succès claires

2. **⚖️ Pondérer vos critères**
   - Qualité vs vitesse vs coût
   - Selon votre contexte business
   - Avec contraintes techniques

3. **🧪 Tester méthodiquement**
   - 2-3 modèles finalistes
   - Sur échantillon représentatif
   - Mesurer latence ET qualité

4. **📈 Projeter à votre échelle**
   - Coûts mensuels réalistes
   - Besoins d'infrastructure
   - Plan de montée en charge

### 🚀 Prochaine Étape

Dans le **notebook final**, nous allons créer votre **matrice de décision personnalisée** qui automatise tout ce processus !

---

**🎉 Félicitations ! Vous savez maintenant tester et comparer les LLM de manière rigoureuse !**